In [38]:
# Importation des bibliothèques

import os
import json
import getpass
import requests

from langchain_mistralai.chat_models import ChatMistralAI
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.prompts import PipelinePromptTemplate




In [39]:
# 🔹 **Configuration API Mistral AI**
MISTRAL_API_KEY = getpass.getpass("Entrez votre clé API Mistral: ")  
MODEL_NAME = "mistral-small-latest"

In [40]:

# 🔹 **Initialisation du modèle**
llm = ChatMistralAI(
    mistral_api_key=MISTRAL_API_KEY,
    model=MODEL_NAME,
    streaming=True,
    temperature=0.6
)

In [41]:
# 🔹 **Historique de la conversation**
conversation_history = []

In [42]:

# 🔹 **Définition des sous-prompts**

prompt_principal = ChatPromptTemplate.from_template(
    """
    Tu es **Crystal Bot**, un **assistant d'orientation professionnelle**.
    Ton objectif est d'aider chaque utilisateur à **clarifier son avenir** en lui proposant **des conseils sur-mesure et motivants**.
    ➤ **Adapte-toi à l’utilisateur** : si indécis, propose des pistes d’exploration.
    ➤ **Si comparaison de plusieurs options**, aide-le à peser le pour et le contre.
    ➤ Ajoute parfois une touche légère de **vision d’avenir**, sans trop en faire.
    """
)

topic_prompt = PromptTemplate.from_template(
    """
    Analyse la question et identifie son domaine principal :
    Question: {input}
    """
)

intent_prompt = PromptTemplate.from_template(
    """
    Analyse la question et identifie l’intention principale de l’utilisateur :
    Question: {input}
    """
)

chatbot_prompt = PromptTemplate.from_template(
    """
    L'utilisateur a demandé : {input}.
    Domaine détecté : {topic} (caché à l'utilisateur)
    Intention détectée : {intent} (caché à l'utilisateur)
    
    ➤ **Si l'intention est "Métiers"** :
        - Propose des métiers adaptés et innovants.
        - Donne une vision inspirante et dynamique.
        
    ➤ **Si l'intention est "Formations"** :
        - Propose des formations adaptées et alternatives.
        - Explique les opportunités de spécialisation.

    ➤ **Si l'intention est "Salaire"** :
        - Explique les différences selon l’expérience et le secteur.
        - Évoque les tendances actuelles.

    ➤ **Si l'intention est "Offres d'emploi"** :
        - Donne des conseils personnalisés sur la recherche d’emploi.
        - Propose une approche proactive.

    ➤ **Si l'intention est "Parcoursup"** :
        - Explique comment optimiser son dossier et lettre de motivation.

    ➤ **Si l'intention est "Autre"** :
        - Propose une réponse originale en fonction du contexte.
    """
)

closing_prompt = PromptTemplate.from_template(
    """
    J’espère que cette réponse **t’a aidé à y voir plus clair !** 🔮  
    **As-tu d’autres questions ?** Je suis là pour explorer toutes les possibilités avec toi.  
    """
)


In [43]:

# 🔹 **Fonction CoT améliorée**
def generate_response_with_cot(user_query):
    """ Génère une réponse en utilisant Chain of Thought et le LLM """
    
    # Étape 1 : Identification du sujet et de l'intention
    topic_output = llm.invoke([HumanMessage(content=topic_prompt.format(input=user_query))]).content
    intent_output = llm.invoke([HumanMessage(content=intent_prompt.format(input=user_query))]).content
    
    # Étape 2 : Génération d'une réponse avec le chatbot_prompt
    chatbot_input = chatbot_prompt.format(input=user_query, topic=topic_output, intent=intent_output)
    chatbot_output = llm.invoke([HumanMessage(content=chatbot_input)]).content

    # Étape 3 : Ajout d'un message de clôture
    closing_output = llm.invoke([HumanMessage(content=closing_prompt.format())]).content

    # ✅ Correction : Retourne un dictionnaire structuré
    return {"response": chatbot_output + "\n" + closing_output}


In [44]:

# 🔹 **Fonction de Refinement améliorée**
def refine_response(chatbot_output, user_query):
    """ Améliore la clarté et la précision de la réponse avec le LLM """
    
    refinement_prompt = f"""
    Améliore la réponse suivante en la rendant plus précise et engageante :
    Contexte : {user_query}
    Réponse initiale : {chatbot_output}
    Réponse améliorée :
    """

    refined_response = llm.invoke([HumanMessage(content=refinement_prompt)]).content
    
    return refined_response


In [36]:

#  **Test utilisateur**
user_query = "Je veux savoir quelles études mènent à une carrière dans l'environnement."
final_response = generate_response_with_cot(user_query)

#  **Affichage de la réponse finale**
print("\n🚀 Réponse optimisée de Crystal IA :")
print(final_response["response"])



🚀 Réponse optimisée de Crystal IA :
Pour répondre à votre question sur les études qui mènent à une carrière dans l'environnement, voici une vue d'ensemble des options éducatives et des parcours possibles :

### Domaines d'études

1. **Sciences de l'environnement** :
   - **Licence (Bachelor)** : Un diplôme de licence en sciences de l'environnement vous donnera une base solide en écologie, biologie, chimie, et gestion des ressources naturelles.
   - **Master** : Un master en sciences de l'environnement peut se spécialiser dans des domaines comme la gestion des déchets, la conservation de la biodiversité, ou la durabilité.

2. **Ingénierie environnementale** :
   - **Licence (Bachelor)** : Ce programme combine des cours en ingénierie avec des cours spécifiques à l'environnement, comme la gestion de l'eau, la qualité de l'air, et les technologies de traitement des déchets.
   - **Master** : Un master en ingénierie environnementale peut se concentrer sur des aspects plus techniques et ava

In [45]:

#  **Test utilisateur**
user_query = " Quels conseils peux tu me donner pour parcoursup."
final_response = generate_response_with_cot(user_query)

# **Affichage de la réponse finale**
print("\n🚀 Réponse de Crystal IA :")
print(final_response["response"])



🚀 Réponse optimisée de Crystal IA :
Pour optimiser votre parcours sur Parcoursup et maximiser vos chances d'admission, voici quelques conseils pratiques :

### 1. **Choisir les Formations**
   - **Évaluez vos intérêts et compétences** : Choisissez des formations qui correspondent à vos passions et à vos aptitudes. Il est important de vous sentir motivé et engagé dans ce que vous étudiez.
   - **Diversifiez vos vœux** : Ne mettez pas tous vos œufs dans le même panier. Faites une liste de formations complémentaires ou alternatives pour augmenter vos chances d'admission.

### 2. **Rédiger une Lettre de Motivation Convaincante**
   - **Personnalisez chaque lettre** : Adaptez votre lettre de motivation à chaque formation. Mentionnez pourquoi vous êtes particulièrement intéressé par cette formation spécifique et comment elle s'aligne avec vos objectifs professionnels.
   - **Mettez en avant vos expériences et compétences** : Parlez de vos expériences professionnelles, de vos stages, de vos 